### Archivo optimizado para la simulación de los eventos de compra de botellas de agua en Medellín

In [0]:
# PASO 1: Imports & Config
import json, random, uuid, time
from pathlib import Path
from datetime import datetime
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import pyarrow
from scipy.stats import truncnorm
from numpy.random import poisson
from shapely.validation import make_valid
import numpy as np

cfg      = json.loads(Path("/Workspace/Users/danielale22rojas@gmail.com/medellin-bigdata-poc/notebooks/1_Ingestion/sim_config.json").read_text()) # Cambiar el nombre de usuario antes de ejecutar
base     = Path(cfg["base_path"])
paths    = cfg["paths"]
interval = cfg["interval_seconds"]
qty_min, qty_max = cfg["quantity_range"]

In [0]:
# PASO 2: Carga de insumos
gdf_neigh = gpd.read_parquet(base/paths["neighborhoods"])       # barrios
mask_geom = gpd.read_parquet(base/paths["city_mask"]).geometry.iloc[0]  # contorno Medellín
df_cust   = pd.read_parquet(base/paths["customers"])            # clientes
df_emp    = pd.read_parquet(base/paths["employees"])            # empleados
# print(f"✅ Barrios: {len(gdf_neigh)} | Clientes: {len(df_cust)} | Empleados: {len(df_emp)}")

In [0]:
# Aplicamos la funcion para corregir la geometría
gdf_neigh["geometry"] = gdf_neigh["geometry"].apply(make_valid)

# Primero corregimos el nombre
gdf_neigh.loc[gdf_neigh["NOMBRE"].isna(), "NOMBRE"] = "ARANJUEZ"

# Luego disolvemos por el nombre
gdf_neigh = gdf_neigh.dissolve(by="NOMBRE", as_index=False)

# Removemos la palabra corregimiento de los nombres
gdf_neigh['NOMBRE'] = gdf_neigh['NOMBRE'].str.replace('CORREGIMIENTO DE ', '', regex=True)

In [0]:
# Diccionario en MAYÚSCULAS (transcribir del DANE)
poblacion = {
    "POPULAR": 120000,
    "SANTA CRUZ": 100000,
    "MANRIQUE": 150000,
    "ARANJUEZ": 110000,
    "CASTILLA": 140000,
    "DOCE DE OCTUBRE": 130000,
    "ROBLEDO": 160000,
    "VILLA HERMOSA": 115000,
    "BUENOS AIRES": 125000,
    "LA CANDELARIA": 90000,
    "LAURELES ESTADIO": 95000,
    "LA AMÉRICA": 110000,
    "SAN JAVIER": 145000,
    "EL POBLADO": 80000,
    "GUAYABAL": 85000,
    "BELÉN": 155000,
    "SAN SEBASTIÁN DE PALMITAS": 20000,
    "SAN CRISTÓBAL": 50000,
    "ALTAVISTA": 30000,
    "SAN ANTONIO DE PRADO": 60000,
    "SANTA ELENA": 40000
}

# Convertir NOMBRE a mayúsculas y mapear población
# gdf_neigh["NOMBRE"] = gdf_neigh["NOMBRE"].str.upper()
gdf_neigh["poblacion"] = gdf_neigh["NOMBRE"].map(poblacion)

# Verificar si hay nulos (barrios sin población asignada)
# print(gdf_neigh[gdf_neigh["poblacion"].isna()][["NOMBRE"]]) # Debe quedar nulo


In [0]:
gdf_neigh.sample(1, weights="poblacion").iloc[0]

In [0]:
# PASO 3: Funciones de muestreo y generación de evento
def sample_point(poly):
    minx,miny,maxx,maxy = poly.bounds
    while True:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p) and mask_geom.contains(p):
            return p

# Como usamos una distribución gausiana truncada, debemos definir los parametros 
# Parámetros de la distribución truncada
# mu = 25      # número promedio de compras por simulación
# sigma = 10   # dispersión
# min_val = qty_min  # mínimo número de compras por intervalo
# max_val = qty_max # máximo aceptado

# # Convertir a valores estandarizados
# min_val = (min_val - mu) / sigma
# max_val = (max_val - mu) / sigma

def gen_event():
    # La asiganción de los barrios segun los pesos de la poblacion
    b  = gdf_neigh.sample(1, weights="poblacion").iloc[0]
    pt = sample_point(b.geometry)
    # Generar cantidad de productos (ej. Poisson proporcional a la población)
    lam = b["poblacion"] / 5000   # escala de λ
    return {
      "order_id":          str(uuid.uuid4()),
      "date":              datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
      "customer_id":       int(df_cust.customer_id.sample(1).iloc[0]),
      "employee_id":       int(df_emp.employee_id.sample(1).iloc[0]),
      # "quantity_products": int(truncnorm(min_val, max_val, loc=mu, scale=sigma).rvs()),
      "quantity_products":  np.random.poisson(lam=max(5, lam)),
      "latitude":          pt.y,
      "longitude":         pt.x,
      "neighborhood":      b["NOMBRE"]
    }


In [0]:
# PASO 4: Preparar carpeta timestamp
ts      = datetime.now().strftime("%Y%m%d_%H%M%S")
out_dir = base/paths["output_dir"]/ts
out_dir.mkdir(parents=True, exist_ok=True)
print("▶️ Carpeta de simulación:", out_dir.name)

In [0]:
# PASO 5 optimizada para prueba rápida:
# Defninimos el número de eventos a generar como una variable aleatoria con distribución normal truncada

# # Parámetros de la distribución truncada
# mu2 = 40      # número promedio de compras por simulación
# sigma2 = 25   # dispersión
# min_val2 = 0  # mínimo número de compras por intervalo
# max_val2 = 70 # máximo aceptado

# # Convertir a valores estandarizados
# a = (min_val2 - mu2) / sigma2
# b = (max_val2 - mu2) / sigma2
# N = int(truncnorm(a, b, loc=mu2, scale=sigma2).rvs())

# Simular con distribución Poisson
N = poisson(lam=30)

# Simular con uniforme
# N = random.randint(0, 30)

# Corremos el bucle para generar los eventos 
for _ in range(N):
    e = gen_event()
    (out_dir/f"{e['order_id']}.json").write_text(json.dumps(e))
print(f"✅ Generados {N} eventos en {out_dir.name}")

In [0]:
# PASO 6: Leer los JSONs y crear DataFrame Spark
files   = list(out_dir.glob("*.json"))
events  = [json.loads(p.read_text()) for p in files]
df_raw  = spark.createDataFrame(events)

In [0]:
# PASO 7: Geo‑join para calcular 'district'  
from shapely.geometry import Point

# 1) Convertir df_raw a pandas para hacer spatial join
pdf = df_raw.toPandas()
pdf = df_raw.toPandas().drop(columns=["neighborhood"])

# 2) Crear GeoDataFrame puntual con lat/lon
pdf["geometry"] = pdf.apply(lambda r: Point(r.longitude, r.latitude), axis=1)
gpdf = gpd.GeoDataFrame(pdf, geometry="geometry", crs=gdf_neigh.crs)

# 3) Spatial join: cada punto recibe el polígono que lo contiene
#    Suponemos gdf_neigh tiene columna 'NOMBRE' con el barrio
gpdf = gpd.sjoin(gpdf, gdf_neigh[["IDENTIFICACION","NOMBRE", "geometry"]], how="left", predicate="within")

# 4) Renombrar la columna resultante y limpiar índices
# gpdf = gpdf.rename(columns={"IDENTIFICACION": "district", "NOMBRE": "neighborhood"}).drop(columns=["index_right"])
gpdf = gpdf.rename(columns={"IDENTIFICACION": "neighborhood", "NOMBRE": "district"}).drop(columns=["index_right"])


# 5) Volver a Spark
df_raw = spark.createDataFrame(gpdf.drop(columns="geometry"))

In [0]:
# PASO 8: Transformar df_raw al esquema Bronze
from pyspark.sql.functions import (
    to_timestamp, date_format,
    year, month, dayofmonth,
    hour, minute, second
)

df_bronze = (
    df_raw
      # 1) Parsear timestamp
      .withColumn("event_ts", to_timestamp("date", "dd/MM/yyyy HH:mm:ss"))
      # 2) Partición diaria en formato ddMMyyyy
      .withColumn("partition_date", date_format("event_ts", "ddMMyyyy"))
      # 3) Desglosar fecha/hora
      .withColumn("event_year",   year("event_ts"))
      .withColumn("event_month",  month("event_ts"))
      .withColumn("event_day",    dayofmonth("event_ts"))
      .withColumn("event_hour",   hour("event_ts"))
      .withColumn("event_minute", minute("event_ts"))
      .withColumn("event_second", second("event_ts"))
      # 4) Renombrar/seleccionar columnas según spec
      .select(
         "partition_date",
         "order_id",
         "neighborhood",
         "customer_id",
         "employee_id",
         "event_ts",
         "event_year","event_month","event_day",
         "event_hour","event_minute","event_second",
         "latitude","longitude",
         "district",
         "quantity_products"
      )
)

# Inspección rápida del resultado
# display(df_bronze.limit(5))
# df_bronze.printSchema()

In [0]:
%sh
# Limpiar la carpeta sim_events
rm -r /Workspace/Users/danielale22rojas@gmail.com/medellin-bigdata-poc/data/sim_events/*

In [0]:
%sql
-- Celda X: Crear el schema de prueba
CREATE DATABASE IF NOT EXISTS poctesting;

In [0]:
# PASO 9: Persistir en Delta como managed table
# Si la tabla no existe, la crea; si existe, hace append.

# 1) Guardar como tabla delta en el metastore
(
  df_bronze
    .write
    .format("delta")
    .mode("append")
    .saveAsTable("poctesting.bronze_events")
)
